In [20]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import folium
from geopy.geocoders import Nominatim
import re

### 장애인보호구역+사회복지시설

In [21]:
# 서울시 사회복지시설 목록 불러오기
welfare = pd.read_csv('./data/서울시 사회복지시설 목록.csv', encoding='cp949')
welfare

,시설명,시설코드,시설종류명(시설유형),시설종류상세명(시설종류),자치구(시)구분,시설장명,시군구코드,시군구명,시설주소,정원(수용인원),현인원,전화번호,우편번호
0,서울꽃동네신내노인요양원,A0001,(노인) 노인요양시설,노인의료복지시설,자치구,최은숙,1126000000,중랑구,서울특별시 중랑구 신내로 194,234.0,234.0,02-490-2609,02052
1,청운양로원,A0002,(노인) 양로시설,노인주거복지시설,자치구,이종명,1111000000,종로구,서울특별시 종로구 비봉길 76 (구기동),57.0,50.0,02-379-9232,03001
2,청운노인요양원,A0003,(노인) 노인요양시설,노인의료복지시설,자치구,이종후,1111000000,종로구,서울특별시 종로구 비봉길 76 (구기동),45.0,45.0,02-3217-0057,03001
3,홍파양로원,A0004,(노인) 양로시설,노인주거복지시설,자치구,김우리,1135000000,노원구,서울특별시 노원구 동일로248길 30 (상계동),44.0,32.0,02-939-0735,01623
4,천사노인요양원,A0007,(노인) 노인요양시설,노인의료복지시설,자치구,김샛별,1150000000,강서구,서울특별시 강서구 강서로45다길 30-22,161.0,151.0,02-2602-2443,07704
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4105,강남발달장애인평생교육센터,Z6325,(장애인) (기타)장애인복지시설,장애인기타,자치구,김미현,1168000000,강남구,"서울특별시 강남구 논현로86길 21, 2층 (역삼동)",24.0,NaN,07043544701,06220
4106,도봉발달장애인 평생교육센터,Z6399,(장애인) (기타)장애인복지시설,장애인기타,자치구,성효진,1132000000,도봉구,"서울특별시 도봉구 마들로 703, 4~6층 (도봉동)",30.0,NaN,029557979,01327
4107,염리종합사회복지관,Z6469,(일반) 사회복지관,일반사회복지시설,자치구,최상진,1144000000,마포구,서울특별시 마포구 대흥로24길 50 (염리동),NaN,NaN,02-3276-1803,04123
4108,서초1인가구지원센터,Z6475,(기타) 기타시설,기타시설,자치구,엄준,1165000000,서초구,"서울특별시 서초구 사평대로 273, 4층 (반포동)",NaN,NaN,0221558283,06544


In [22]:
# 사회복지시설 목록 중 필요한 열만 뽑기
welfare_df = welfare[['시군구명','시설명','시설주소','시설종류명(시설유형)','정원(수용인원)','현인원']]
welfare_df

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원
0,중랑구,서울꽃동네신내노인요양원,서울특별시 중랑구 신내로 194,(노인) 노인요양시설,234.0,234.0
1,종로구,청운양로원,서울특별시 종로구 비봉길 76 (구기동),(노인) 양로시설,57.0,50.0
2,종로구,청운노인요양원,서울특별시 종로구 비봉길 76 (구기동),(노인) 노인요양시설,45.0,45.0
3,노원구,홍파양로원,서울특별시 노원구 동일로248길 30 (상계동),(노인) 양로시설,44.0,32.0
4,강서구,천사노인요양원,서울특별시 강서구 강서로45다길 30-22,(노인) 노인요양시설,161.0,151.0
...,...,...,...,...,...,...
4105,강남구,강남발달장애인평생교육센터,"서울특별시 강남구 논현로86길 21, 2층 (역삼동)",(장애인) (기타)장애인복지시설,24.0,NaN
4106,도봉구,도봉발달장애인 평생교육센터,"서울특별시 도봉구 마들로 703, 4~6층 (도봉동)",(장애인) (기타)장애인복지시설,30.0,NaN
4107,마포구,염리종합사회복지관,서울특별시 마포구 대흥로24길 50 (염리동),(일반) 사회복지관,NaN,NaN
4108,서초구,서초1인가구지원센터,"서울특별시 서초구 사평대로 273, 4층 (반포동)",(기타) 기타시설,NaN,NaN


In [23]:
# 시설종류명(시설유형)에 '장애'가 포함되어 있는 것만 뽑기 + 인덱스 재정렬
df_welfare = welfare_df[welfare_df['시설종류명(시설유형)'].str.contains('장애')]
df_welfare = df_welfare.reset_index(drop=True)
df_welfare

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원
0,송파구,송파구방이복지관,서울특별시 송파구 백제고분로37길 4 (석촌동)(석촌동),(장애인) 장애인복지관,160.0,160.0
1,송파구,서울시각장애인복지관,서울특별시 송파구 삼학사로19길 27,(장애인) 장애인복지관,0.0,0.0
2,동작구,서울시립발달장애인복지관,서울특별시 동작구 여의대방로20길 33-0395(보라매공원내),(장애인) 장애인복지관,752.0,752.0
3,강남구,청음복지관,서울특별시 강남구 봉은사로50길 6,(장애인) 장애인복지관,NaN,345.0
4,마포구,마포장애인종합복지관,서울특별시 마포구 성산로4길 353층(마포장애인종합복지관)(성산동),(장애인) 장애인복지관,NaN,3743.0
...,...,...,...,...,...,...
679,종로구,종로발달장애인평생교육센터,서울특별시 종로구 종로17길 8 (종로2가),(장애인) (기타)장애인복지시설,30.0,NaN
680,영등포구,영등포구 발달장애인평생교육센터,"서울특별시 영등포구 문래북로 1055~7층, 어울림센터 (당산동1가)",(장애인) (기타)장애인복지시설,NaN,NaN
681,강남구,강남발달장애인평생교육센터,"서울특별시 강남구 논현로86길 21, 2층 (역삼동)",(장애인) (기타)장애인복지시설,24.0,NaN
682,도봉구,도봉발달장애인 평생교육센터,"서울특별시 도봉구 마들로 703, 4~6층 (도봉동)",(장애인) (기타)장애인복지시설,30.0,NaN


In [24]:
# 시설 주소의 null값 확인
df_welfare['시설주소'].isnull().sum()  # 5개의 결측값 존재
df_welfare[df_welfare['시설주소'].isnull()]  # 결측값 존재하는 행 확인

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원
67,노원구,쉼터,NaN,(장애인) 중증장애인거주시설,70.0,57.0
302,강동구,사랑쉼터의집,NaN,(장애인) 장애유형별거주시설,29.0,21.0
639,성북구,성북피해장애인쉼터,NaN,(장애인) 피해장애인쉼터,8.0,NaN
646,서초구,단양노인장애인복지관,NaN,(장애인) 장애인근로사업장,NaN,NaN
647,서초구,금강직업재활시설,NaN,(장애인) 장애인근로사업장,NaN,NaN


In [25]:
# 시설주소가 없는 시설에 시설주소 추가
df_welfare['시설주소'][df_welfare['시설명']=='사랑쉼터의집'] = '서울특별시 강동구 상암로79길 24'
df_welfare['시설주소'][df_welfare['시설명']=='성북피해장애인쉼터'] = '서울특별시 성북구 화랑로 134'

/var/folders/km/7c_dj80d1gs40kyd912vcjq80000gn/T/ipykernel_38902/3902850333.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_welfare['시설주소'][df_welfare['시설명']=='사랑쉼터의집'] = '서울특별시 강동구 상암로79길 24'
/var/folders/km/7c_dj80d1gs40kyd912vcjq80000gn/T/ipykernel_38902/3902850333.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_welfare['시설주소'][df_welfare['시설명']=='성북피해장애인쉼터'] = '서울특별시 성북구 화랑로 134'


In [26]:
# 시설주소를 못 찾는 시설은 제거
df_welfare = df_welfare.drop([67, 646, 647])
df_welfare = df_welfare.reset_index(drop=True)
df_welfare

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원
0,송파구,송파구방이복지관,서울특별시 송파구 백제고분로37길 4 (석촌동)(석촌동),(장애인) 장애인복지관,160.0,160.0
1,송파구,서울시각장애인복지관,서울특별시 송파구 삼학사로19길 27,(장애인) 장애인복지관,0.0,0.0
2,동작구,서울시립발달장애인복지관,서울특별시 동작구 여의대방로20길 33-0395(보라매공원내),(장애인) 장애인복지관,752.0,752.0
3,강남구,청음복지관,서울특별시 강남구 봉은사로50길 6,(장애인) 장애인복지관,NaN,345.0
4,마포구,마포장애인종합복지관,서울특별시 마포구 성산로4길 353층(마포장애인종합복지관)(성산동),(장애인) 장애인복지관,NaN,3743.0
...,...,...,...,...,...,...
676,종로구,종로발달장애인평생교육센터,서울특별시 종로구 종로17길 8 (종로2가),(장애인) (기타)장애인복지시설,30.0,NaN
677,영등포구,영등포구 발달장애인평생교육센터,"서울특별시 영등포구 문래북로 1055~7층, 어울림센터 (당산동1가)",(장애인) (기타)장애인복지시설,NaN,NaN
678,강남구,강남발달장애인평생교육센터,"서울특별시 강남구 논현로86길 21, 2층 (역삼동)",(장애인) (기타)장애인복지시설,24.0,NaN
679,도봉구,도봉발달장애인 평생교육센터,"서울특별시 도봉구 마들로 703, 4~6층 (도봉동)",(장애인) (기타)장애인복지시설,30.0,NaN


In [27]:
# 시설명과 시군구명이 같은 행 확인
df_welfare[df_welfare.duplicated(subset=['시군구명', '시설명'], keep=False)]  # 총 3개의 행(가장 앞쪽에 있는 행) 외의 다른 데이터는 제거할 예정

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원
610,노원구,킨더윌언어심리전문센터,"서울특별시 노원구 상계로23다길 13-8, 1층 107-1,7,8호 (상계동, 노원...",(장애인) 장애인재활치료시설,150.0,NaN
628,중랑구,소담,"서울특별시 중랑구 상봉로 69, 5층 (면목동)",(장애인) 장애인보호작업장,15.0,NaN
635,은평구,스프링샤인,서울특별시 은평구 갈현로11길 30 (구산동),(장애인) 장애인보호작업장,20.0,12.0
636,중랑구,소담,"서울특별시 중랑구 상봉로69 (면목동, 한솔빌딩 5층)",(장애인) 장애인보호작업장,15.0,NaN
646,노원구,킨더윌언어심리전문센터,"서울특별시 노원구 상계로23다길 13-8107호 (상계동, 노원 아이파크)",(장애인) 장애인재활치료시설,NaN,0.0
653,중랑구,소담,서울특별시 중랑구 상봉로 695층 (면목동),(장애인) 장애인보호작업장,15.0,NaN
655,은평구,스프링샤인,서울특별시 은평구 갈현로11길 30 (구산동),(장애인) 장애인보호작업장,20.0,10.0


In [28]:
# '시설명'과 '시군구명'이 같은 행을 찾아냅니다.
duplicated_rows = df_welfare[df_welfare.duplicated(subset=['시군구명', '시설명'], keep=False)]

# 첫 세 행을 제외한 나머지 행의 인덱스를 가져옴
drop_indices = duplicated_rows.index[3:]

# 해당 인덱스의 행을 제거
df_wel = df_welfare.drop(drop_indices)
df_wel = df_wel.reset_index(drop=True)
df_wel

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원
0,송파구,송파구방이복지관,서울특별시 송파구 백제고분로37길 4 (석촌동)(석촌동),(장애인) 장애인복지관,160.0,160.0
1,송파구,서울시각장애인복지관,서울특별시 송파구 삼학사로19길 27,(장애인) 장애인복지관,0.0,0.0
2,동작구,서울시립발달장애인복지관,서울특별시 동작구 여의대방로20길 33-0395(보라매공원내),(장애인) 장애인복지관,752.0,752.0
3,강남구,청음복지관,서울특별시 강남구 봉은사로50길 6,(장애인) 장애인복지관,NaN,345.0
4,마포구,마포장애인종합복지관,서울특별시 마포구 성산로4길 353층(마포장애인종합복지관)(성산동),(장애인) 장애인복지관,NaN,3743.0
...,...,...,...,...,...,...
672,종로구,종로발달장애인평생교육센터,서울특별시 종로구 종로17길 8 (종로2가),(장애인) (기타)장애인복지시설,30.0,NaN
673,영등포구,영등포구 발달장애인평생교육센터,"서울특별시 영등포구 문래북로 1055~7층, 어울림센터 (당산동1가)",(장애인) (기타)장애인복지시설,NaN,NaN
674,강남구,강남발달장애인평생교육센터,"서울특별시 강남구 논현로86길 21, 2층 (역삼동)",(장애인) (기타)장애인복지시설,24.0,NaN
675,도봉구,도봉발달장애인 평생교육센터,"서울특별시 도봉구 마들로 703, 4~6층 (도봉동)",(장애인) (기타)장애인복지시설,30.0,NaN


- 주소를 위도, 경도로 변환하기
- sheet -> 확장프로그램 -> geocoding -> 시설주소
- xlsx 파일 정리함

In [29]:
# hwp - 서울시 장애인 보호구역 목록
protect = pd.DataFrame({'자치구':['강서구','강서구','동작구','강남구','강남구','송파구','송파구','강동구','강동구','강동구'],
                        '시설명' :['늘푸른집','늘푸른나무복지관','서울시립남부장애인복지관','다니엘의집4호','세움복지관','신아재활원','임마누엘 집','암사재활원','사랑쉼터의 집','홀트강동복지관'],
                        '위치':['금낭화로26가길 130','허준로5길 37','여의대방로20나길 40','도산대로4길 18-6','광평로60길22','양산로8길 17','양산로10길 26','올림픽로 864','상암로79길 24','아리수로93길 41'],
                        '지정일':[2015,2014,2022,2015,2023,2012,2012,2013,2017,2021]})
protect

,자치구,시설명,위치,지정일
0,강서구,늘푸른집,금낭화로26가길 130,2015
1,강서구,늘푸른나무복지관,허준로5길 37,2014
2,동작구,서울시립남부장애인복지관,여의대방로20나길 40,2022
3,강남구,다니엘의집4호,도산대로4길 18-6,2015
4,강남구,세움복지관,광평로60길22,2023
5,송파구,신아재활원,양산로8길 17,2012
6,송파구,임마누엘 집,양산로10길 26,2012
7,강동구,암사재활원,올림픽로 864,2013
8,강동구,사랑쉼터의 집,상암로79길 24,2017
9,강동구,홀트강동복지관,아리수로93길 41,2021


In [30]:
# 좌표 데이터를 hwp 파일 데이터프레임에 합치기.
protect_l = protect.join(latlog)
protect_l

NameError: name 'latlog' is not defined

In [ ]:
# 임마누엘 집 좌표 추가
protect_l['longitude'][6] = 127.1467
protect_l['latitude'][6] = 37.4916

NameError: name 'protect_l' is not defined

In [ ]:
protect_l

NameError: name 'protect_l' is not defined

장애인보호구역 시각화

In [ ]:
# 서울의 중심 위도, 경도
center_lat, center_lon = 37.5665, 126.9780

# 지도 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=11)

# 각 장소를 지도에 표시
for idx, row in protect_l.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['시설명']).add_to(m)

# 지도 표시
m

In [ ]:
# 복지시설 시각화

In [31]:
# 보호구역 + 복지시설 시각화


In [68]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Selenium 웹 드라이버 초기화
driver = webdriver.Chrome()

# 웹사이트 열기
url = "https://taas.koroad.or.kr/tsm/tmb/mmg/initTrafficSafeMap.do?menuId=TSM_TMB"
# driver.get(url)

# 검색창에 '강남구' 입력
driver.get(url)
search_box = driver.find_element(By.ID, 'poiSearchWord')  # 검색창의 id에 따라 변경해야 합니다.
search_box.clear()
search_box.send_keys('강남구')

# 검색 버튼 클릭
search_button = driver.find_element(By.CLASS_NAME, 'btn-search')  # 검색 버튼의 Xpath에 따라 변경해야 합니다.
search_button.click()

# 페이지 로드를 기다림
time.sleep(5)

occurrence_count = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div[1]/table/tbody/tr[1]/td').text
occurrence_count = int(occurrence_count)
print(occurrence_count)

# Selenium 종료
driver.quit()

13


In [74]:
df_wel['시설명'][:10]


0          송파구방이복지관
1        서울시각장애인복지관
2      서울시립발달장애인복지관
3             청음복지관
4        마포장애인종합복지관
5             다운복지관
6        원광장애인종합복지관
7    성프란치스꼬장애인종합복지관
8           성모자애복지관
9               대린원
Name: 시설명, dtype: object

In [69]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Selenium 웹 드라이버 초기화
driver = webdriver.Chrome()

# 웹사이트 열기
url = "https://taas.koroad.or.kr/tsm/tmb/mmg/initTrafficSafeMap.do?menuId=TSM_TMB"
# driver.get(url)

# 검색창에 '강남구' 입력
driver.get(url)
search_box = driver.find_element(By.ID, 'poiSearchWord')  # 검색창의 id에 따라 변경해야 합니다.
search_box.clear()
search_box.send_keys('강남구')

# 검색 버튼 클릭
search_button = driver.find_element(By.CLASS_NAME, 'btn-search')  # 검색 버튼의 Xpath에 따라 변경해야 합니다.
search_button.click()

# 페이지 로드를 기다림
time.sleep(5)

occurrence_count = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div[1]/table/tbody/tr[1]/td').text
occurrence_count = int(occurrence_count)
print(occurrence_count)

# Selenium 종료
driver.quit()

13


In [111]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def get_occurrence_count(facility_name):
    # Selenium 웹 드라이버 초기화
    driver = webdriver.Chrome()

    # 웹사이트 열기
    url = "https://taas.koroad.or.kr/tsm/tmb/mmg/initTrafficSafeMap.do?menuId=TSM_TMB"
    driver.get(url)

    # 검색창에 시설명 입력
    search_box = driver.find_element(By.ID, 'poiSearchWord')
    search_box.clear()
    search_box.send_keys(facility_name)

    # 검색 버튼 클릭
    search_button = driver.find_element(By.CLASS_NAME, 'btn-search')
    search_button.click()

    # 페이지 로드를 기다림
    time.sleep(5)

    # 결과에서 사고건수 가져오기
    try:
        occurrence_count = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div[1]/table/tbody/tr[1]/td').text
        occurrence_count = int(occurrence_count)
    except Exception as e:
        print(f"Error retrieving occurrence count for {facility_name}: {e}")
        occurrence_count = None

    # Selenium 종료
    driver.quit()

    return occurrence_count

# df_wel에 '사고건수' 열 추가
df_wel['사고건수'] = df_wel['시설명'].iloc[600:].apply(lambda x: get_occurrence_count(x) if x else None)


In [90]:
df_wel1 = df_wel.iloc[:100]

In [93]:
df_wel1

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원,사고건수
0,송파구,송파구방이복지관,서울특별시 송파구 백제고분로37길 4 (석촌동)(석촌동),(장애인) 장애인복지관,160.0,160.0,23.0
1,송파구,서울시각장애인복지관,서울특별시 송파구 삼학사로19길 27,(장애인) 장애인복지관,0.0,0.0,13.0
2,동작구,서울시립발달장애인복지관,서울특별시 동작구 여의대방로20길 33-0395(보라매공원내),(장애인) 장애인복지관,752.0,752.0,21.0
3,강남구,청음복지관,서울특별시 강남구 봉은사로50길 6,(장애인) 장애인복지관,NaN,345.0,47.0
4,마포구,마포장애인종합복지관,서울특별시 마포구 성산로4길 353층(마포장애인종합복지관)(성산동),(장애인) 장애인복지관,NaN,3743.0,24.0
...,...,...,...,...,...,...,...
95,강동구,보금자리,서울특별시 강동구 동남로85길 154층,(장애인) 장애인공동생활가정,4.0,3.0,32.0
96,강동구,하늘자리공동생활가정,서울특별시 강동구 동남로85길 152층,(장애인) 장애인공동생활가정,4.0,4.0,7.0
97,도봉구,주바라기해피홈,서울특별시 도봉구 도봉로169가길 45도봉동,(장애인) 장애인주간보호시설,20.0,20.0,18.0
98,노원구,동천하우스1,서울특별시 노원구 동일로237바길 8301호 (상계동),(장애인) 장애인공동생활가정,4.0,4.0,0.0


In [94]:
df_wel2 = df_wel.iloc[100:200]

In [97]:
df_wel2

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원,사고건수
100,노원구,해맑음보호작업시설,서울특별시 노원구 동일로 1097(공릉동),(장애인) 장애인보호작업장,30.0,30.0,35.0
101,강북구,강북구수어통역센터,"서울특별시 강북구 도봉로 3885층 (번동,테마빌딩)",(장애인) 수화통역센터,46.0,7115.0,77.0
102,노원구,그루터기공동생활가정,서울특별시 노원구 동일로191가길 26-0남청파인힐 101동 203호,(장애인) 장애인공동생활가정,4.0,4.0,19.0
103,관악구,관악구수어통역센터,서울특별시 관악구 쑥고개로 128관악구 장애인 행복센터2층,(장애인) 수화통역센터,44.0,100.0,24.0
104,도봉구,도봉구수어통역센터,서울특별시 도봉구 마들로13길 153옥산빌딩 4층 (창동),(장애인) 수화통역센터,0.0,0.0,4.0
...,...,...,...,...,...,...,...
195,송파구,삼전주간보호시설,서울특별시 송파구 백제고분로32길 35삼전종합복지관 3층 삼전주간보호시설,(장애인) 장애인주간보호시설,15.0,15.0,10.0
196,구로구,브니엘의 집,서울특별시 구로구 구로중앙로21길 22-1,(장애인) 장애유형별거주시설,25.0,13.0,30.0
197,노원구,이수경하우스,서울특별시 노원구 덕릉로122길 15-4202호 (상계동),(장애인) 장애인공동생활가정,4.0,4.0,0.0
198,송파구,송파시각장애인축구장,서울특별시 송파구 올림픽로 474,(장애인) 장애인체육시설,0.0,0.0,36.0


In [99]:
df_wel3 = df_wel.iloc[200:300]

In [100]:
df_wel3

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원,사고건수
200,금천구,임마누엘시흥공동체,서울특별시 금천구 독산로50마길 19-10성은빌라 201호 (시흥동),(장애인) 장애인공동생활가정,4.0,4.0,5.0
201,금천구,임마누엘독산공동체,"서울특별시 금천구 시흥대로118길 44302호 (독산동, 정우파크빌라)",(장애인) 장애인공동생활가정,4.0,4.0,0.0
202,양천구,임마누엘신월공동체,"서울특별시 양천구 오목로11길 9나동 501호 (신월동, 우정쉐르빌)",(장애인) 장애인공동생활가정,4.0,4.0,38.0
203,양천구,임마누엘신정공동체,"서울특별시 양천구 오목로 148-34301호 (신정동, 오성이지빌)",(장애인) 장애인공동생활가정,4.0,3.0,31.0
204,구로구,임마누엘개봉공동체,"서울특별시 구로구 고척로16가길 35102호 (개봉동, 선경빌라)",(장애인) 장애인공동생활가정,4.0,4.0,37.0
...,...,...,...,...,...,...,...
295,강남구,하상주간보호센터,서울특별시 강남구 개포로 613 (개포동),(장애인) 장애인주간보호시설,12.0,12.0,0.0
296,마포구,라온홈,서울특별시 마포구 잔다리로9길 21105동 501호,(장애인) 장애인공동생활가정,4.0,4.0,14.0
297,용산구,한벗장애인주간보호시설,서울특별시 용산구 효창원로69길 42-3한벗회관 2층,(장애인) 장애인주간보호시설,10.0,10.0,7.0
298,용산구,행복이가득한집,"서울특별시 용산구 후암로28라길 9201호 (후암동, 예원빌라)",(장애인) 장애인공동생활가정,3.0,3.0,7.0


In [103]:
df_wel4 = df_wel.iloc[300:400]

In [104]:
df_wel4

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원,사고건수
300,관악구,실로암배움울타리,서울특별시 관악구 조원로12길 66-02층,(장애인) 장애인공동생활가정,6.0,5.0,32.0
301,강동구,사랑쉼터의집,서울특별시 강동구 상암로79길 24,(장애인) 장애유형별거주시설,29.0,21.0,17.0
302,양천구,베다니장애인주간보호시설,서울특별시 양천구 남부순환로65길 12-2 (신월동),(장애인) 장애인주간보호시설,10.0,13.0,0.0
303,용산구,가브리엘의집,서울특별시 용산구 후암로35길 22-23(후암동),(장애인) 중증장애인거주시설,29.0,29.0,9.0
304,노원구,동천일리하우스,서울특별시 노원구 노원로18길 41(하계1동),(장애인) 중증장애인거주시설,60.0,39.0,11.0
...,...,...,...,...,...,...,...
395,강남구,충현의집6호,서울특별시 강남구 삼성로67길 8-7401호 (대치동)(대치동),(장애인) 장애인공동생활가정,1.0,1.0,0.0
396,은평구,한마음1호공동생활가정,서울특별시 은평구 통일로89길 5-73층,(장애인) 장애인공동생활가정,4.0,4.0,0.0
397,강서구,기쁜우리체육센터,서울특별시 강서구 허준로 65,(장애인) 장애인체육시설,NaN,300.0,5.0
398,관악구,에스알씨(SRC)재활체육관,경기도 광주시 초월읍 경수길 25에스알씨재활체육관,(장애인) 장애인체육시설,NaN,1000.0,0.0


In [106]:
df_wel5 = df_wel.iloc[400:500]

In [107]:
df_wel5

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원,사고건수
400,관악구,에스알씨(SRC)부설의원,서울특별시 관악구 보라매로 44보라매동,(장애인) 장애인의료재활시설,NaN,100.0,0.0
401,관악구,에스알씨(SRC)재활병원,경기도 광주시 경수길 25에스알씨재활병원,(장애인) 장애인의료재활시설,NaN,700.0,5.0
402,은평구,서울재활병원,서울특별시 은평구 갈현로11길 30-0,(장애인) 장애인의료재활시설,350.0,350.0,14.0
403,도봉구,도봉장애인보호작업시설,서울특별시 도봉구 마들로 664-172층,(장애인) 장애인보호작업장,30.0,19.0,6.0
404,송파구,마라보호작업장,서울특별시 송파구 송이로34길 43-1,(장애인) 장애인보호작업장,20.0,22.0,17.0
...,...,...,...,...,...,...,...
495,중구,서울특별시중구장애인복지관,서울특별시 중구 퇴계로 460중구종합복지센터 4층,(장애인) 장애인복지관,0.0,190.0,13.0
496,금천구,금천구립장애인보호작업장,"서울특별시 금천구 한내로 69-15금천한내복지관내 (독산동, 주공13단지아파트)",(장애인) 장애인보호작업장,32.0,32.0,0.0
497,강북구,너나울공동생활가정,"서울특별시 강북구 인수봉로60가길 24-7201호 (수유동, 다우빌라)",(장애인) 장애인공동생활가정,4.0,4.0,0.0
498,강북구,참사랑너나울주간보호센터,서울특별시 강북구 한천로143길 391층 (수유동),(장애인) 장애인주간보호시설,15.0,12.0,67.0


In [109]:
df_wel6 = df_wel.iloc[500:600]

In [110]:
df_wel6

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원,사고건수
500,강서구,강서구희망센터,"서울특별시 강서구 화곡로31길 25지층(화곡동), 서울특별시 강서구 강서로 34길 ...",(장애인) 장애인보호작업장,34.0,38.0,35.0
501,강서구,늘푸른집,서울특별시 강서구 금낭화로26가길 1301층,(장애인) 장애인단기거주시설,10.0,10.0,25.0
502,강남구,하상점자도서관,서울특별시 강남구 개포로 613,(장애인) 장애인복지관,30.0,30.0,14.0
503,강남구,광림주간보호센터,서울특별시 강남구 압구정로22길 21웨슬리관 2층,(장애인) 장애인주간보호시설,12.0,10.0,4.0
504,은평구,빛나눔보호작업장,서울특별시 은평구 수색로 390-29 (수색동),(장애인) 장애인보호작업장,23.0,31.0,8.0
...,...,...,...,...,...,...,...
595,금천구,한결엠보호작업장,"서울특별시 금천구 가산디지털1로 131, B206호 (가산동, BYC하이시티지식산업센터)",(장애인) 장애인보호작업장,10.0,10.0,0.0
596,송파구,우리그룹홈,"서울특별시 송파구 거마로3길 5, 가동 102호 (거여동, 대진아트빌라)",(장애인) 장애인공동생활가정,4.0,4.0,7.0
597,구로구,블루나래,"서울특별시 구로구 디지털로 272, 12층 17호 (구로동, 한신아이티타워)",(장애인) 장애인보호작업장,10.0,13.0,36.0
598,서대문구,영광시각장애인주간보호센터,서울특별시 서대문구 세무서10길 54 (홍제동),(장애인) 장애인주간보호시설,15.0,14.0,0.0


In [113]:
df_wel7 = df_wel.iloc[600:]

In [114]:
df_wel7

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원,사고건수
600,구로구,모두누리 직업재활센터,서울특별시 구로구 디지털로27길 24506호(벽산디지털밸리 1차) (구로동),(장애인) 장애인보호작업장,20.0,24.0,35.0
601,금천구,기쁨나무,서울특별시 금천구 시흥대로80길 19 (독산동),(장애인) 장애인보호작업장,20.0,17.0,25.0
602,강동구,홀트주간보호센터,서울특별시 강동구 아리수로93길 41 (강일동),(장애인) 장애인주간보호시설,10.0,10.0,27.0
603,강서구,주은공동생활가정,"서울특별시 강서구 강서로5라길 138, 2층 (화곡동)",(장애인) 장애인공동생활가정,4.0,3.0,0.0
604,구로구,은하수,"서울특별시 구로구 고척로52마길 21-11, 1층 (개봉동)",(장애인) 장애인공동생활가정,4.0,4.0,134.0
...,...,...,...,...,...,...,...
672,종로구,종로발달장애인평생교육센터,서울특별시 종로구 종로17길 8 (종로2가),(장애인) (기타)장애인복지시설,30.0,NaN,59.0
673,영등포구,영등포구 발달장애인평생교육센터,"서울특별시 영등포구 문래북로 1055~7층, 어울림센터 (당산동1가)",(장애인) (기타)장애인복지시설,NaN,NaN,34.0
674,강남구,강남발달장애인평생교육센터,"서울특별시 강남구 논현로86길 21, 2층 (역삼동)",(장애인) (기타)장애인복지시설,24.0,NaN,82.0
675,도봉구,도봉발달장애인 평생교육센터,"서울특별시 도봉구 마들로 703, 4~6층 (도봉동)",(장애인) (기타)장애인복지시설,30.0,NaN,21.0


In [115]:
df_wel

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원,사고건수
0,송파구,송파구방이복지관,서울특별시 송파구 백제고분로37길 4 (석촌동)(석촌동),(장애인) 장애인복지관,160.0,160.0,NaN
1,송파구,서울시각장애인복지관,서울특별시 송파구 삼학사로19길 27,(장애인) 장애인복지관,0.0,0.0,NaN
2,동작구,서울시립발달장애인복지관,서울특별시 동작구 여의대방로20길 33-0395(보라매공원내),(장애인) 장애인복지관,752.0,752.0,NaN
3,강남구,청음복지관,서울특별시 강남구 봉은사로50길 6,(장애인) 장애인복지관,NaN,345.0,NaN
4,마포구,마포장애인종합복지관,서울특별시 마포구 성산로4길 353층(마포장애인종합복지관)(성산동),(장애인) 장애인복지관,NaN,3743.0,NaN
...,...,...,...,...,...,...,...
672,종로구,종로발달장애인평생교육센터,서울특별시 종로구 종로17길 8 (종로2가),(장애인) (기타)장애인복지시설,30.0,NaN,59.0
673,영등포구,영등포구 발달장애인평생교육센터,"서울특별시 영등포구 문래북로 1055~7층, 어울림센터 (당산동1가)",(장애인) (기타)장애인복지시설,NaN,NaN,34.0
674,강남구,강남발달장애인평생교육센터,"서울특별시 강남구 논현로86길 21, 2층 (역삼동)",(장애인) (기타)장애인복지시설,24.0,NaN,82.0
675,도봉구,도봉발달장애인 평생교육센터,"서울특별시 도봉구 마들로 703, 4~6층 (도봉동)",(장애인) (기타)장애인복지시설,30.0,NaN,21.0


In [117]:
dfs_concat = [df_wel1, df_wel2, df_wel3, df_wel4, df_wel5, df_wel6, df_wel7]
total_accident__welfare_facilities = pd.concat(dfs_concat, ignore_index=True)

In [118]:
total_accident__welfare_facilities

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원,사고건수
0,송파구,송파구방이복지관,서울특별시 송파구 백제고분로37길 4 (석촌동)(석촌동),(장애인) 장애인복지관,160.0,160.0,23.0
1,송파구,서울시각장애인복지관,서울특별시 송파구 삼학사로19길 27,(장애인) 장애인복지관,0.0,0.0,13.0
2,동작구,서울시립발달장애인복지관,서울특별시 동작구 여의대방로20길 33-0395(보라매공원내),(장애인) 장애인복지관,752.0,752.0,21.0
3,강남구,청음복지관,서울특별시 강남구 봉은사로50길 6,(장애인) 장애인복지관,NaN,345.0,47.0
4,마포구,마포장애인종합복지관,서울특별시 마포구 성산로4길 353층(마포장애인종합복지관)(성산동),(장애인) 장애인복지관,NaN,3743.0,24.0
...,...,...,...,...,...,...,...
672,종로구,종로발달장애인평생교육센터,서울특별시 종로구 종로17길 8 (종로2가),(장애인) (기타)장애인복지시설,30.0,NaN,59.0
673,영등포구,영등포구 발달장애인평생교육센터,"서울특별시 영등포구 문래북로 1055~7층, 어울림센터 (당산동1가)",(장애인) (기타)장애인복지시설,NaN,NaN,34.0
674,강남구,강남발달장애인평생교육센터,"서울특별시 강남구 논현로86길 21, 2층 (역삼동)",(장애인) (기타)장애인복지시설,24.0,NaN,82.0
675,도봉구,도봉발달장애인 평생교육센터,"서울특별시 도봉구 마들로 703, 4~6층 (도봉동)",(장애인) (기타)장애인복지시설,30.0,NaN,21.0


In [119]:
total_accident__welfare_facilities.to_excel('total_accident_welfare_facilities.xlsx', index=False)